After the setup, one can make an active buy/sell indicator which allows an investor to follow investor trading while trading the S&P500. 

In [1]:
import requests
import pandas as pd
from datetime import datetime

def fetch_insider_data():
    url = 'http://openinsider.com/ps_data.csv'
    response = requests.get(url)
    
    if response.status_code != 200:
        raise Exception("Failed to fetch data from the server")

    data = response.text.splitlines()
    columns = ['filing_date', 'num_purchases', 'num_sales', 'val_purchases', 'val_sales']
    
    data_list = []
    for line in data:
        if line.startswith('Date'):
            continue  # Skip the header line
        if not line.startswith('#') and line.strip():
            values = line.split('\t')
            if len(values) > 1 and values[0] != '000000':
                try:
                    filing_date = datetime.strptime(values[0], '%y%m%d')
                    num_purchases = int(values[1])
                    num_sales = int(values[2])
                    val_purchases = int(values[3])
                    val_sales = int(values[4])
                    data_list.append([filing_date, num_purchases, num_sales, val_purchases, val_sales])
                except ValueError:
                    continue  # Skip lines with invalid date format

    df = pd.DataFrame(data_list, columns=columns)
    return df

def get_latest_purchases_sales():
    insider_data = fetch_insider_data()
    latest_date = insider_data['filing_date'].max()
    latest_data = insider_data[insider_data['filing_date'] == latest_date]
    latest_record = latest_data.iloc[0]
    return latest_record['filing_date'], latest_record['num_purchases'], latest_record['num_sales']

latest_date, num_purchases, num_sales = get_latest_purchases_sales()
purchases_sales_ratio = num_purchases / num_sales if num_sales != 0 else float('inf')
print(f"Latest Date: {latest_date.strftime('%Y-%m-%d')}")
print(f"Number of Purchases: {num_purchases}")
print(f"Number of Sales: {num_sales}")
print(f"Purchases to Sales Ratio: {purchases_sales_ratio:.2f}")


Latest Date: 2024-08-21
Number of Purchases: 54
Number of Sales: 211
Purchases to Sales Ratio: 0.26


Fully functioning up to date indicator below

In [4]:
import requests
import pandas as pd
from datetime import datetime

def fetch_insider_data():
    url = 'http://openinsider.com/ps_data.csv'
    response = requests.get(url)
    
    if response.status_code != 200:
        raise Exception("Failed to fetch data from the server")

    data = response.text.splitlines()
    columns = ['filing_date', 'num_purchases', 'num_sales', 'val_purchases', 'val_sales']
    
    data_list = []
    for line in data:
        if line.startswith('Date'):
            continue  # Skip the header line
        if not line.startswith('#') and line.strip():
            values = line.split('\t')
            if len(values) > 1 and values[0] != '000000':
                try:
                    filing_date = datetime.strptime(values[0], '%y%m%d')
                    num_purchases = int(values[1])
                    num_sales = int(values[2])
                    val_purchases = int(values[3])
                    val_sales = int(values[4])
                    data_list.append([filing_date, num_purchases, num_sales, val_purchases, val_sales])
                except ValueError:
                    continue  # Skip lines with invalid date format

    df = pd.DataFrame(data_list, columns=columns)
    return df

def get_latest_and_statistics():
    insider_data = fetch_insider_data()
    
    # Latest data
    latest_date = insider_data['filing_date'].max()
    latest_data = insider_data[insider_data['filing_date'] == latest_date]
    latest_total_purchases = latest_data['num_purchases'].sum()
    latest_total_sales = latest_data['num_sales'].sum()
    latest_total_combined = latest_total_purchases + latest_total_sales

    # Median data
    median_num_purchases = insider_data['num_purchases'].median()
    median_num_sales = insider_data['num_sales'].median()
    
    # Median purchases to sales ratio
    insider_data['purchases_sales_ratio'] = insider_data['num_purchases'] / insider_data['num_sales']
    median_purchases_sales_ratio = insider_data['purchases_sales_ratio'].median()

    # Total combined purchases and sales
    total_combined = insider_data['num_purchases'].sum() + insider_data['num_sales'].sum()
    median_combined = (insider_data['num_purchases'] + insider_data['num_sales']).median()

    return (latest_date, latest_total_purchases, latest_total_sales, latest_total_combined,
            median_num_purchases, median_num_sales, median_purchases_sales_ratio,
            total_combined, median_combined)

latest_date, latest_total_purchases, latest_total_sales, latest_total_combined, median_num_purchases, median_num_sales, median_purchases_sales_ratio, total_combined, median_combined = get_latest_and_statistics()
purchases_sales_ratio = latest_total_purchases / latest_total_sales if latest_total_sales != 0 else float('inf')

# Determine buy, sell, and hold signals
buy_signal = purchases_sales_ratio >= 1
sell_signal = purchases_sales_ratio <= 0.20
hold_signal = not buy_signal and not sell_signal

print(f"Latest Date: {latest_date.strftime('%Y-%m-%d')}")
print(f"Total Number of Purchases Today: {latest_total_purchases}")
print(f"Total Number of Sales Today: {latest_total_sales}")
print(f"Total Purchases and Sales Today: {latest_total_combined}")
print(f"Purchases to Sales Ratio Today: {purchases_sales_ratio:.2f}")
print()
print(f"Median Number of Purchases: {median_num_purchases}")
print(f"Median Number of Sales: {median_num_sales}")
print(f"Median Combined Purchases and Sales: {median_combined}")
print(f"Median Purchases to Sales Ratio: {median_purchases_sales_ratio:.2f}")
print()
print(f"Buy Signal (>=1): {'Yes' if buy_signal else 'No'}")
print(f"Sell Signal (<=0.20): {'Yes' if sell_signal else 'No'}")
print(f"Hold Signal (Otherwise): {'Yes' if hold_signal else 'No'}")


Latest Date: 2024-08-21
Total Number of Purchases Today: 54
Total Number of Sales Today: 211
Total Purchases and Sales Today: 265
Purchases to Sales Ratio Today: 0.26

Median Number of Purchases: 73.0
Median Number of Sales: 163.0
Median Combined Purchases and Sales: 249.0
Median Purchases to Sales Ratio: 0.42

Buy Signal (>=1): No
Sell Signal (<=0.20): No
Hold Signal (Otherwise): Yes
